In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
# @vla: custom parse options and quantize function
from lavis.args_parser import parse_args
from lavis.quantize import quantize

In [6]:
# from lavis.args_parser import parse_args
# from lavis.quantize import *
args = vars(parse_args())
args

usage: ipykernel_launcher.py [-h] --cfg-path CFG_PATH
                             [--options OPTIONS [OPTIONS ...]]
                             [--visual-encoder-block-modules [{qkv,proj,fc1,fc2} ...]]
                             [--visual-encoder-block-indices [VISUAL_ENCODER_BLOCK_INDICES ...]]
                             [--visual-encoder-block-weight-bits {1,2,4,6,8}]
                             [--qformer-layer-indices [QFORMER_LAYER_INDICES ...]]
                             [--qformer-self-attention-modules [{query,key,value,dense} ...]]
                             [--qformer-self-attention-weight-bits {1,2,4,6,8}]
                             [--qformer-cross-attention-modules [{query,key,value,dense} ...]]
                             [--qformer-cross-attention-weight-bits {1,2,4,6,8}]
                             [--qformer-text-ff-intermediate {dense}]
                             [--qformer-text-ff-output {dense}]
                             [--qformer-text-ff-weight

SystemExit: 2

In [5]:
cfg = Config(parse_args())
cfg

In [6]:
task = tasks.setup_task(cfg)
task

In [15]:
model = task.build_model(cfg)
model

Position interpolate from 16x16 to 26x26


Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-38): 39 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1408, out_features=4224, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1408, out_features=1408, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )


In [16]:
model

Blip2Qformer(
  (visual_encoder): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-38): 39 x Block(
        (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1408, out_features=4224, bias=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1408, out_features=1408, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1408, out_features=6144, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )


In [4]:
quantize(model, args)

NameError: name 'model' is not defined

In [19]:
model.visual_encoder.blocks

ModuleList(
  (0-38): 39 x Block(
    (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
    (attn): Attention(
      (qkv): NBitLinearDynamic(in_features=1408, out_features=4224, bias=False)
      (attn_drop): Dropout(p=0.0, inplace=False)
      (proj): NBitLinearDynamic(in_features=1408, out_features=1408, bias=True)
      (proj_drop): Dropout(p=0.0, inplace=False)
    )
    (drop_path): Identity()
    (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
    (mlp): Mlp(
      (fc1): NBitLinearDynamic(in_features=1408, out_features=6144, bias=True)
      (act): GELU(approximate='none')
      (fc2): NBitLinearDynamic(in_features=6144, out_features=1408, bias=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
  )
)

In [21]:
for name, module in model.Qformer.named_children():
    print(name)

bert
cls


In [27]:
model.Qformer.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30523, 768)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (crossattention): BertAttention(
         